<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Bag of Visual Words - Assignment 2

     Instructor: Kris Kitani                       TAs:Paritosh (Lead), Rawal, Yan, Zen, Wen-Hsuan, Qichen

In [1]:
import nbimporter
import numpy as np
import scipy.ndimage
from skimage import io
import skimage.transform
import os,time
import util
import multiprocessing
import threading
import queue
import torch
import torchvision
import torchvision.transforms

In [23]:
from skimage.transform import resize
import scipy.spatial.distance
from scipy.spatial.distance import cdist

## For Autograding P4, ensure uploading `trained_conf_matrix.npy` and `trained_system_deep.npz`.

### Deep Learning Features - An Alternative to ``Bag of Words``

As we have discussed in class, another powerful method for scene classification in computer vision is the employment of convolutional neural networks (CNNs) - sometimes referred to generically as $deep learning$. It is important to understand how previously trained (pretrained) networks can be used as another form of feature extraction, and how they relate to classical Bag of Words (BoW) features. We will be covering details on how one chooses the network architecture and training procedures later in the course. For this question, however, we will be asking you to deal with the VGG-16 pretrained network. VGG-16 is a pretrained Convolutional Neural Network (CNN) that has been trained on approximately 1.2 million images from the ImageNet Dataset (``http://image-net.org/index``) by the Visual Geometry Group (VGG) at University of Oxford. The model can classify images into a 1000 object categories (e.g. keyboard, mouse, coffee mug, pencil).

One lesson we want you to take away from this exercise is to understand the effectiveness of $deep$ $features$ for general classification tasks within computer vision - even when those features have been previously trained on a different dataset (i.e. ImageNet) and task (i.e. object recognition). 

#### Extracting Deep Features

To complete this question, you need to install the ``torchvision`` library from Pytorch, a popular Python-based deep learning library.
If you are using the Anaconda package manager (``https://www.anaconda.com/download/``), this can be done with the following command:
```
            conda install pytorch torchvision -c pytorch
```
To check that you have installed it correctly, make sure that you can ``import torch`` in a Python interpreter without errors.
Please refer to ``https://pytorch.org/`` for more detailed installation instructions. 

#### Q4.1.1 (25 Points)

We want to extract out deep features corresponding to the convolutional layers of theVGG-16 network.  In this problem, we will use the trained weights from the VGG network directly, but implement our own operations. To load the network, use the line
```
        vgg16 = torchvision.models.vgg16(pretrained=True).double()
```
followed by ``vgg16.eval()``
The latter line ensures that the VGG-16 network is in evaluation mode, not training mode.

path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"

image = io.imread(path_img)

image = image.astype('float')/255

vgg16 = torchvision.models.vgg16(pretrained=True).double()

vgg16.eval()

We want you to complete a function that is able to output $VGG-16$ network outputs at the **fc7** layer in
```
    def extract_deep_feature(x,vgg16_weights):
```
where ``x`` refers to the input image and ``vgg16_weights`` is a structure containing the CNN's network parameters. In this function you will need to write sub-functions ``multichannel_conv2d``, ``relu``, ``max_pool2d``, and ``linear`` corresponding to the fundamental elements of the CNN: multi-channel convolution, rectified linear units (ReLU), max pooling, and fully-connected weighting.

We have provided a helper function ``util.get_VGG16_weights()`` that extracts the weight parameters of VGG-16 and its meta information. The returned variable is a numpy array of shape $L\times 3$, where $L$ is the number of layers in VGG-16. The first column of each row is a string indicating the layer type. The second/third columns may contain the learned weights and biases, or other meta-information (\eg kernel size of max-pooling). Please refer to the function docstring for details.

In order to build the ``extract_deep_feature`` function, you should run a for-loop through each layer index until layer **fc7**, which corresponds to **the second linear layer** (Refer to VGG structure to see where **fc7** is). **Remember**: the output of the preceding layer should be passed through as an input to the next.

Details on the sub-functions needed for the ``extract_deep_feature`` function can be found below.

Please use ``scipy.ndimage.convolve`` and ``numpy`` functions to implement these functions instead of using pytorch. Please keep speed in mind when implementing your function, for example, using double for loop over pixels is not a good idea.

``multichannel_conv2d(x,weight,bias)``:

a function that will perform multi-channel 2D convolution which can be defined as follows, 

\begin{equation}
\mathbf{y}^{(j)} = \sum_{k=1}^{K} \begin{bmatrix} \mathbf{x}^{(k)} * \mathbf{h}^{(j,k)} \end{bmatrix} + \mathbf{b}[j] 
\end{equation}

where $*$ denotes $2D$ convolution, $\mathbf{x} = \{ \mathbf{x}^{(k)} \}_{k=1}^{K}$ is our vectorized $K$-channel input signal, $\mathbf{h} = \{ \mathbf{h}^{(j,k)} \}_{k=1,j=1}^{K,J}$ is our $J \times K$ set of vectorized convolutional filters and $\mathbf{r} = \{ \mathbf{y}^{(j)} \}_{j=1}^{J}$ is our $J$ channel vectorized output response. Further, unlike traditional single-channel convolution CNNs often append a bias vector $\mathbf{b}$ whose $J$ elements are added to the $J$ channels of the output response. 

To implement ``multichannel_conv2d``, you can use the Scipy convolution function directly with for loops to cycle through the filters and channels (``scipy.ndimage.convolve()``). All the necessary details concerning the number of filters ($J$), number of channels ($K$), filter weights ($\mathbf{h}$) and biases ($\mathbf{b}$) can be inferred from the shapes/dimensions of the weights and biases. Notice that pytorch's convolution function actually does correlation, so to get similar answer as pytroch with scipy, you need to flip the kernel on both axes using ``np.flip()``.

In [71]:
def multichannel_conv2d(x,weight,bias):
    '''
    Performs multi-channel 2D convolution.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim,kernel_size,kernel_size)
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * feat: numpy.ndarray of shape (H,W,output_dim)
    '''
    print("\n Input x shape: ", x.shape, "\n")
    h, w, input_dims = x.shape
    output_dims = weight.shape[0]
    final_res = np.zeros((h, w, output_dims))
    '''
    HINTS:
    1.> for 2D convolution we need to use np.fliplr and np.flipud
    2.> can use scipy.ndimage.convolve with the flipped kernel
    3.> don't forget to add the bias
    '''
    for j in range(output_dims):
      
        output = np.zeros((h,w))
        for k in range(input_dims):
            conv = weight[j,k,:,:]
            filt = np.flipud(np.fliplr(conv))
            output += scipy.ndimage.convolve(x[:,:,k], filt, mode ='constant', cval =0 )
        final_res[:,:, j] = output + bias[j]
    # YOUR CODE HERE
    #raise NotImplementedError()
    print("\n Input x final_res: ", final_res.shape, "\n")
    return final_res

```relu(x)```:

a function that shall perform the Rectified Linear Unit (ReLU) which can be defined mathematically as,

\begin{equation}
\mbox{ReLU}(x) = \max_{x}(x, 0)
\end{equation}

and is applied independently to each element of the matrix/vector $x$ passed to it.

In [72]:
def relu(x):
    '''
    Rectified linear unit.

    [input]
    * x: numpy.ndarray

    [output]
    * y: numpy.ndarray
    '''
    y = np.maximum(x, 0)
    # YOUR CODE HERE
    #raise NotImplementedError()
    print("\n Input y shape: ", y.shape, "\n")
    return y


``max_pool2d(x,size):``

a function that shall perform max pooling over $x$ using a receptive field of size $size$ $\times$ $size$ (we assume a square receptive field here for simplicity).

  If the function receives a multi-channel input, then it should apply the max pooling operation across each input channel independently.
  
(Hint: making use of smart array indexing can drastically speed up the code.)

In [73]:
def max_pool2d(x,size):
    '''
    2D max pooling operation.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * size: pooling receptive field

    [output]
    * y: numpy.ndarray of shape (H/size,W/size,input_dim)
    '''
    print("\n Input x shape: ", x.shape, "\n")
    h, w, dims = x.shape
    pooled_arr = np.empty((h//size, w//size, dims))
    for i in range(h//size):
        for j in range(w//size):
            for k in range(dims):
                pooled_window= x[i*size : (i+1)*size, j*size : (j+1)*size,k] 
                pooled_arr[i,j,k] = np.max(pooled_window, axis=(0,1))
    '''
    HINTS:
    1.> estimate the shape you need to apply the pooling operation.
    2.> We can smart fill the padding with np.nan and then use np.nanmax to select the max (avoiding nan)
    3.> We can input the grid (start_x:end_x, start_y:end_y, dim) as smart array indexing to np.nanmax
    '''
    # YOUR CODE HERE
    #raise NotImplementedError()
    print("\n Input x shape: ", pooled_arr.shape, "\n")
    return pooled_arr

``linear(x,W,b):``

a function that will compute a node vector where each element is a linear combination of the input nodes, written as

\begin{equation}
\mathbf{y}[j] = \sum_{k=1}^{K}\mathbf{W}[j,k] \mathbf{x}[k] + \mathbf{b}[j] 
\end{equation}

or more succinctly in vector form as $\mathbf{y} = \mathbf{W} \mathbf{x} + \mathbf{b}$ - where $\mathbf{x}$ is the $(K \times 1)$ input node vector, $\mathbf{W}$ is the $(J \times K)$ weight matrix, $\mathbf{b}$ is the $(J \times 1)$ bias vector and $\mathbf{y}$ is the~$(J \times 1)$ output node vector.

You should not need for-loops to implement this function.

In [85]:
def linear(x,W,b):
    '''
    Fully-connected layer.

    [input]
    * x: numpy.ndarray of shape (input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim)
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * y: numpy.ndarray of shape (output_dim)
    '''
    print("\n Input x shape: ", x.shape, "\n")
    #x = np.reshape(x, (-1, 1))
    y = np.dot( W, x) + b
    # YOUR CODE HERE
    #raise NotImplementedError()
    print("\n Input y shape: ", y.shape, "\n")
    return y

You should ignore all ``DropoutLayer`` you encounter; they're functional only during the training phase.

VGG-16 assumes that all input imagery to the network is resized to $224 \times 224$ with the three color channels preserved (use ``skimage.transform.resize()`` to do this before passing any imagery to the network). And be sure to normalize the image using suggested mean and std before extracting the feature:
```
                                        mean=[0.485,0.456,0.406]}
                                        std=[0.229,0.224,0.225]}
```

In [86]:
def preprocess_image(image):
    '''
    Preprocesses the image to load into the prebuilt network.

    [input]
    * image: numpy.ndarray of shape (H,W,3)

    [output]
    * image_processed: torch.array of shape (3,H,W)
    '''
    
    # ----- TODO -----
    
    if(len(image.shape) == 2):
        image = np.stack((image, image, image), axis=-1)

    if(image.shape == 3 and image.shape[2] == 1):
        image = np.concatenate((image, image, image), axis=-1)

    if(image.shape[2] == 4):
        image = image[:, :, 0:3]
   
    mean=np.asarray([0.485,0.456,0.406])
    std=np.asarray([0.229,0.224,0.225])
    M = np.tile(mean,(224, 224,1))
    Std = np.tile(std,(224, 224,1))
    '''
    HINTS:
    1.> Resize the image (look into skimage.transform.resize)
    2.> normalize the image
    3.> convert the image from numpy to torch
    '''
    img = resize(image, (224, 224, image.shape[2]), anti_aliasing=True)
    img2 = (img - M)//Std 
    img2 = np.transpose(img2 , (2, 0, 1))
    image_processed = torch.from_numpy(img2)
    # YOUR CODE HERE
    #raise NotImplementedError()
    return image_processed

For efficiency you should check that each sub-function is working properly before putting them all together - otherwise it will be hard to track any errors. To compare your implementation with pytroch, you should compare the extracted features between your ``extract_deep_feature``  and the pre-trained VGG-16 network.  ``evaluate_deep_extractor`` should come in handy in comparing the result of the two extracted features.

In [87]:
def extract_deep_feature(x, vgg16_weights):
    '''
    Extracts deep features from the given VGG-16 weights.

    [input]
    * x: numpy.ndarray of shape (H,W,3)
    * vgg16_weights: list of shape (L,3)

    [output]
    * feat: numpy.ndarray of shape (K)
    '''
    feat = np.copy(x) 
    linear_count =0
    n_layers= 34
    for layer in range(n_layers):
        if (vgg16_weights[layer][0] == "conv2d"):
            print(feat.shape)
            feat = multichannel_conv2d(feat,vgg16_weights[layer][1],vgg16_weights[layer][2])
            print(feat.shape)
            print("\n conv2d \n")
        if (vgg16_weights[layer][0] == "relu"):
            
            feat = relu(feat)
            print(feat.shape)
            print("\n relu \n")
        if (vgg16_weights[layer][0] == "maxpool2d"):
            #print(feat.shape)
            feat = max_pool2d(feat,vgg16_weights[layer][1])
            print("\n maxpool2d \n")
            print(feat.shape)
        if (vgg16_weights[layer][0] == "linear"):
            #print(feat.shape)
            if len(feat.shape) == 3:
                feat = np.transpose(feat , (2, 0 , 1)).flatten()
            feat = linear(feat,vgg16_weights[layer][1],vgg16_weights[layer][2])
            print(feat.shape)
            print("\n linear \n")
            linear_count +=1
            if linear_count == 2:
                break
        
    
    
    
    
    
    #feat = np.copy(x) 
    # YOUR CODE HERE
    #raise NotImplementedError()
    return feat

In [88]:
def evaluate_deep_extractor(img, vgg16):
    '''
    Evaluates the deep feature extractor for a single image.

    [input]
    * image: numpy.ndarray of shape (H,W,3)
    * vgg16: prebuilt VGG-16 network.

    [output]
    * diff: difference between the two feature extractor's result
    '''
    
    vgg16_weights = util.get_VGG16_weights()
    img_torch = preprocess_image(img)
    
    feat = extract_deep_feature(np.transpose(img_torch.numpy(), (1,2,0)), vgg16_weights)
    
    with torch.no_grad():
        vgg_classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-3])
        vgg_feat_feat = vgg16.features(img_torch[None, ])
        vgg_feat_feat = vgg_classifier(vgg_feat_feat.flatten())
    
    return np.sum(np.abs(vgg_feat_feat.numpy() - feat))

In [89]:
# path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"

# image = io.imread(path_img)

# image = image.astype('float')/255

# vgg16 = torchvision.models.vgg16(pretrained=True).double()

# vgg16.eval()

# evaluate_deep_extractor(image, vgg16)

(224, 224, 3)

 Input x shape:  (224, 224, 3) 


 Input x final_res:  (224, 224, 64) 

(224, 224, 64)

 conv2d 


 Input y shape:  (224, 224, 64) 

(224, 224, 64)

 relu 

(224, 224, 64)

 Input x shape:  (224, 224, 64) 


 Input x final_res:  (224, 224, 64) 

(224, 224, 64)

 conv2d 


 Input y shape:  (224, 224, 64) 

(224, 224, 64)

 relu 


 Input x shape:  (224, 224, 64) 


 Input x shape:  (112, 112, 64) 


 maxpool2d 

(112, 112, 64)
(112, 112, 64)

 Input x shape:  (112, 112, 64) 


 Input x final_res:  (112, 112, 128) 

(112, 112, 128)

 conv2d 


 Input y shape:  (112, 112, 128) 

(112, 112, 128)

 relu 

(112, 112, 128)

 Input x shape:  (112, 112, 128) 


 Input x final_res:  (112, 112, 128) 

(112, 112, 128)

 conv2d 


 Input y shape:  (112, 112, 128) 

(112, 112, 128)

 relu 


 Input x shape:  (112, 112, 128) 


 Input x shape:  (56, 56, 128) 


 maxpool2d 

(56, 56, 128)
(56, 56, 128)

 Input x shape:  (56, 56, 128) 


 Input x final_res:  (56, 56, 256) 

(56, 56, 256)

3.0453417565468044e-12

### Building a Visual Recognition System: Revisited

We want to compare how useful deep features are in a visual recognition system. Since the speed of the function ``` scipy.ndimage.convolve``` is not ideal, you can use the pytroch VGG-16 network directly (refer to the helper function ```evaluate_deep_extractor``` on how to use the pre-trained network as feature extractor).

#### Q4.2.1 (5 Points Autograder + WriteUp):
Implement the functions
```
                    def build_recognition_system(vgg16):
```
and
```
                    def eval_recognition_system(vgg16)}
```
both of which takes the pretrained VGG-16 network as the input arguments. 

The former function should produce ``trained_system_deep.npz`` as the output. 

Included will be:
* $features$ : a $N \times  K$ matrix containing all the deep features of the $N$ training images in the data set.
* $labels$ : an $N$ vector containing the labels of each of the images. ($features[i]$ will correspond to label $labels[i]$).

The latter function should produce the confusion matrix, as with the previous question.

Instead of using the histogram intersection similarity, write a function to just use the negative Euclidean distance (as larger values are more similar).

**Report the confusion matrix and accuracy for your results in your write-up. Can you comment in your writeup on whether the results are better or worse than classical BoW - why do you think that is?**

In [102]:
def get_image_feature(args):
    '''
    Extracts deep features from the prebuilt VGG-16 network.
    This is a function run by a subprocess.
    [input]
    * i: index of training image
    * image_path: path of image file
    * vgg16: prebuilt VGG-16 network.
    
    [output]
    * feat: evaluated deep feature
    '''
    i, image_path, vgg16 = args
    image = io.imread(image_path)
    image = image.astype('float')/255
    vgg16_weights = util.get_VGG16_weights()
    img_torch = preprocess_image(image)
        
    
    with torch.no_grad():
        vgg_classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-3])
        vgg_feat_feat = vgg16.features(img_torch[None, ])
        vgg_feat_feat = vgg_classifier(vgg_feat_feat.flatten())
    
    feat = vgg_feat_feat.numpy()
    '''
    HINTS:
    1.> Think along the lines of evaluate_deep_extractor
    '''
    # ----- TODO -----
    # YOUR CODE HERE
    #raise NotImplementedError()
    return [i,feat]

In [46]:
def build_recognition_system(vgg16, num_workers=2):
    '''
    Creates a trained recognition system by generating training features from all training images.

    [input]
    * vgg16: prebuilt VGG-16 network.
    * num_workers: number of workers to process in parallel

    [saved]
    * features: numpy.ndarray of shape (N,K)
    * labels: numpy.ndarray of shape (N)
    '''
    args =[]
    train_data = np.load("./data/train_data.npz", allow_pickle=True)
    image_names = train_data['files']
    labels = train_data['labels']
    num_images = image_names.shape[0]
    features= np.zeros((0 , 4096))
    for i in range(num_images):
        #print(image_names[i])
        full_image_path = './data/' + image_names[i]
        args =[i, full_image_path, vgg16]
        j, ordered_features = get_image_feature(args)
        ordered_features = np.asarray(ordered_features)
        #print("Shape of Ordered Feature: ", ordered_features.shape)
        ordered_features = np.reshape(ordered_features,(1, -1))
        #print("Shape of Ordered Feature: ", ordered_features.shape)
        #print("Shape of Feature: ", features.shape)
        features = np.vstack((features, ordered_features))
        print("Shape of Feature: ", features.shape)
    '''
    HINTS:
    1.> Similar approach as Q1.2.2 and Q3.1.1 (create an 
    list and use multiprocessing)
    2.> Keep track of the order in which input is given to multiprocessing
    '''
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    
    '''
    HINTS:
    1.> reorder the features to their correct place as input
    '''
    # YOUR CODE HERE
    #raise NotImplementedError()
    print("done", features.shape)
    
    np.savez('trained_system_deep.npz', features=features, labels=labels)

In [47]:
#build_recognition_system(vgg16, num_workers=2)

Shape of Feature:  (1, 4096)
Shape of Feature:  (2, 4096)
Shape of Feature:  (3, 4096)
Shape of Feature:  (4, 4096)
Shape of Feature:  (5, 4096)
Shape of Feature:  (6, 4096)
Shape of Feature:  (7, 4096)
Shape of Feature:  (8, 4096)
Shape of Feature:  (9, 4096)
Shape of Feature:  (10, 4096)
Shape of Feature:  (11, 4096)
Shape of Feature:  (12, 4096)
Shape of Feature:  (13, 4096)
Shape of Feature:  (14, 4096)
Shape of Feature:  (15, 4096)
Shape of Feature:  (16, 4096)
Shape of Feature:  (17, 4096)
Shape of Feature:  (18, 4096)
Shape of Feature:  (19, 4096)
Shape of Feature:  (20, 4096)
Shape of Feature:  (21, 4096)
Shape of Feature:  (22, 4096)
Shape of Feature:  (23, 4096)
Shape of Feature:  (24, 4096)
Shape of Feature:  (25, 4096)
Shape of Feature:  (26, 4096)
Shape of Feature:  (27, 4096)
Shape of Feature:  (28, 4096)
Shape of Feature:  (29, 4096)
Shape of Feature:  (30, 4096)
Shape of Feature:  (31, 4096)
Shape of Feature:  (32, 4096)
Shape of Feature:  (33, 4096)
Shape of Feature:  

Shape of Feature:  (269, 4096)
Shape of Feature:  (270, 4096)
Shape of Feature:  (271, 4096)
Shape of Feature:  (272, 4096)
Shape of Feature:  (273, 4096)
Shape of Feature:  (274, 4096)
Shape of Feature:  (275, 4096)
Shape of Feature:  (276, 4096)
Shape of Feature:  (277, 4096)
Shape of Feature:  (278, 4096)
Shape of Feature:  (279, 4096)
Shape of Feature:  (280, 4096)
Shape of Feature:  (281, 4096)
Shape of Feature:  (282, 4096)
Shape of Feature:  (283, 4096)
Shape of Feature:  (284, 4096)
Shape of Feature:  (285, 4096)
Shape of Feature:  (286, 4096)
Shape of Feature:  (287, 4096)
Shape of Feature:  (288, 4096)
Shape of Feature:  (289, 4096)
Shape of Feature:  (290, 4096)
Shape of Feature:  (291, 4096)
Shape of Feature:  (292, 4096)
Shape of Feature:  (293, 4096)
Shape of Feature:  (294, 4096)
Shape of Feature:  (295, 4096)
Shape of Feature:  (296, 4096)
Shape of Feature:  (297, 4096)
Shape of Feature:  (298, 4096)
Shape of Feature:  (299, 4096)
Shape of Feature:  (300, 4096)
Shape of

Shape of Feature:  (534, 4096)
Shape of Feature:  (535, 4096)
Shape of Feature:  (536, 4096)
Shape of Feature:  (537, 4096)
Shape of Feature:  (538, 4096)
Shape of Feature:  (539, 4096)
Shape of Feature:  (540, 4096)
Shape of Feature:  (541, 4096)
Shape of Feature:  (542, 4096)
Shape of Feature:  (543, 4096)
Shape of Feature:  (544, 4096)
Shape of Feature:  (545, 4096)
Shape of Feature:  (546, 4096)
Shape of Feature:  (547, 4096)
Shape of Feature:  (548, 4096)
Shape of Feature:  (549, 4096)
Shape of Feature:  (550, 4096)
Shape of Feature:  (551, 4096)
Shape of Feature:  (552, 4096)
Shape of Feature:  (553, 4096)
Shape of Feature:  (554, 4096)
Shape of Feature:  (555, 4096)
Shape of Feature:  (556, 4096)
Shape of Feature:  (557, 4096)
Shape of Feature:  (558, 4096)
Shape of Feature:  (559, 4096)
Shape of Feature:  (560, 4096)
Shape of Feature:  (561, 4096)
Shape of Feature:  (562, 4096)
Shape of Feature:  (563, 4096)
Shape of Feature:  (564, 4096)
Shape of Feature:  (565, 4096)
Shape of

Shape of Feature:  (799, 4096)
Shape of Feature:  (800, 4096)
Shape of Feature:  (801, 4096)
Shape of Feature:  (802, 4096)
Shape of Feature:  (803, 4096)
Shape of Feature:  (804, 4096)
Shape of Feature:  (805, 4096)
Shape of Feature:  (806, 4096)
Shape of Feature:  (807, 4096)
Shape of Feature:  (808, 4096)
Shape of Feature:  (809, 4096)
Shape of Feature:  (810, 4096)
Shape of Feature:  (811, 4096)
Shape of Feature:  (812, 4096)
Shape of Feature:  (813, 4096)
Shape of Feature:  (814, 4096)
Shape of Feature:  (815, 4096)
Shape of Feature:  (816, 4096)
Shape of Feature:  (817, 4096)
Shape of Feature:  (818, 4096)
Shape of Feature:  (819, 4096)
Shape of Feature:  (820, 4096)
Shape of Feature:  (821, 4096)
Shape of Feature:  (822, 4096)
Shape of Feature:  (823, 4096)
Shape of Feature:  (824, 4096)
Shape of Feature:  (825, 4096)
Shape of Feature:  (826, 4096)
Shape of Feature:  (827, 4096)
Shape of Feature:  (828, 4096)
Shape of Feature:  (829, 4096)
Shape of Feature:  (830, 4096)
Shape of

In [48]:
# d = np.load('trained_system_deep.npz')
# d['features']

array([[-0.95326526, -1.1852169 , -0.55441987, ..., -1.61662685,
         0.60368137, -0.58638455],
       [-3.18252807, -4.01928829, -6.52714168, ..., -1.59881208,
        -1.88452777,  0.44608051],
       [-0.5219948 , -1.5655562 , -1.6615287 , ..., -2.89904303,
        -0.90029161, -0.50085174],
       ...,
       [-1.77008454, -1.48252616, -2.18571849, ..., -0.57815144,
        -0.48343413,  0.90129614],
       [-2.60673711, -3.94138553, -8.51294627, ..., -5.87363296,
        -2.53563973, -2.07253576],
       [-2.09022075, -0.49212311, -2.33429464, ..., -0.53872879,
         0.06377819,  0.592439  ]])

In [76]:

def helper_func(args):
    # YOUR CODE HERE
    #raise NotImplementedError()
    i, image_path, vgg16, trained_features, train_labels = args
    _ , feat = get_image_feature([i, image_path, vgg16])
    #print(feat.shape)
    feat_all = np.tile(feat, (trained_features.shape[0], 1))
    #print(feat_all.shape)
    #print(trained_features.shape)
    sum = np.sum((feat_all - trained_features)**2, axis =1)
    #print("\nshape of sum : " ,sum.shape)
    idx = np.argmin(sum, axis = 0)
    #print("\n idx : ", idx)
    
    pred_label = train_labels[idx]
    #print("\n Pred label: ", pred_label)
    return [i, pred_label]


def evaluate_recognition_system(vgg16, num_workers=2):
    '''
    Evaluates the recognition system for all test images and returns the confusion matrix.

    [input]
    * vgg16: prebuilt VGG-16 network.
    * num_workers: number of workers to process in parallel

    [output]
    * conf: numpy.ndarray of shape (8,8)
    * accuracy: accuracy of the evaluated system
    '''
    '''
    HINTS:
    (1) Students can write helper functions (in this cell) to use multi-processing
    '''
    test_data = np.load("./data/test_data.npz", allow_pickle=True)

    # ----- TODO -----
    conf_matrix = np.zeros((8,8))
    trained_system = np.load("trained_system_deep.npz", allow_pickle=True)
    image_names = test_data['files']
    test_labels = test_data['labels']

    trained_features = trained_system['features']
    train_labels = trained_system['labels']
    num_images = image_names.shape[0]
    #print("Trained features shape: ", trained_features.shape)
    #print("Trained label shape: ", train_labels.shape)
    #print(train_labels)
    for i in range(num_images):
        full_image_path = './data/' + image_names[i]
        #print("Test label: ", test_labels[i])
        args = [i, full_image_path, vgg16, trained_features, train_labels]
    
        j , pred_label = helper_func(args)
        conf_matrix[test_labels[i], pred_label] += 1
    
    accuracy = (np.trace(conf_matrix)/ np.sum(conf_matrix))*100
    print(" \naccuracy: ", accuracy)
    '''
    HINTS:
    1.> [Important] Can write a helper function in this cell of jupyter notebook for multiprocessing
    
    2.> Helper function will compute the vgg features for test image (get_image_feature) and find closest
        matching feature from trained_features.
    
    3.> Since trained feature is of shape (N,K) -> smartly repeat the test image feature N times (bring it to
        same shape as (N,K)). Then we can simply compute distance in a vectorized way.
    
    4.> Distance here can be sum over (a-b)**2
    
    5.> np.argmin over distance can give the closest point
    '''
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    #print("Predicted labels shape: ", ordered_labels.shape)
    
    '''
    HINTS:
    1.> Compute the confusion matrix (8x8)
    '''
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    np.save("./trained_conf_matrix.npy",conf_matrix)
    return conf_matrix, accuracy
    # pass

In [77]:
#evaluate_recognition_system(vgg16, num_workers=2)

 
accuracy:  92.5


(array([[14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., 17.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0., 22.,  0.,  0.,  0.,  0.,  3.],
        [ 1.,  0.,  1., 24.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 12.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  2., 22.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0., 20.,  0.],
        [ 0.,  1.,  0.,  1.,  0.,  0.,  0., 17.]]),
 92.5)